In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
from xgcm import Grid
import numpy as np
import pandas as pd
from scipy import signal

import IPython.display
import cmocean as cm
import sys, os, glob
import warnings
warnings.simplefilter("ignore") 

from IPython.display import Image

from dask.distributed import Client

from matplotlib.patches import Rectangle

In [ ]:
# Load workers:
client = Client(n_workers=7)
client

## Testing

In [ ]:
base = '/g/data/e14/rmh561/croco/'
base_data = base + 'archive/'
base_analysis = base + '/analysis/'

os.chdir(base_analysis + 'PAC12_75_cpl')
sys.path.append(base_analysis + 'PAC12_75_cpl/Processing_Scripts')
sys.path.append(base_analysis + 'PAC12_75_cpl/Processing_Scripts/Tracer_balance_code_LMaillard_v2')

In [ ]:
# Load custom scripts/tools:
import pac12_tools as ptools
import R_tools_fort as toolsF

In [ ]:
# Reload (if needed):
import importlib
importlib.reload(ptools)

# Test heat budget code:

In [ ]:
data_day = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_day.nc').rename({'time_counter':'time'})
data_mon = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_mon.nc').rename({'time_counter':'time'})
data_dayTIW = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_day_TIW.nc').rename({'time_counter':'time'})
data_grd = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_grd.nc')

In [ ]:
L1, M1 = 1, 147
[Mm,Lm] = np.shape(data_dayTIW.temp.isel(time=0).isel(s_rho_TIW=0))
N_TIW = np.shape(data_dayTIW.s_rho_TIW)
N = np.shape(data_mon.s_rho)
u = data_dayTIW.u.isel(x_TIW_u=slice(1,-1)).isel(time=0)
v = data_dayTIW.v.isel(y_TIW_v=slice(1,-1)).isel(time=0)
h = data_grd.h.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))
zeta = data_day.zeta.isel(x_rho=slice(L1,L1+Lm),y_rho=slice(M1,M1+Mm)).isel(time=0)
hc = data_grd.hc
Cs_r = data_grd.Cs_r
Cs_w = data_grd.Cs_w
pm = data_grd.pm.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))
pn = data_grd.pn.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))

In [ ]:
z_r, z_w = toolsF.zlevs(h,zeta,hc,Cs_r,Cs_w)
w = toolsF.get_omega(np.transpose(u.values, (2, 1, 0)),np.transpose(v.values, (2, 1, 0)),np.transpose(z_r[:,:,39:],(1,0,2)),np.transpose(z_w[:,:,39:],(1,0,2)),pm.T,pn.T)#,lm=1298,mm=198,n=36)

# Testing high-pass filter processing scripts

In [ ]:
# Testing high-pass filter stuff:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'
file_out = 'test.nc'
filt_width = 12.

data = xr.open_dataset(file_in,chunks={'time_counter':1}).rename({'time_counter':'time'})
data = ptools.create_coords_CROCO(data)
DT = len(data.time)

In [ ]:
# Testing high-pass filter stuff:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_mon.nc'
file_out = 'test.nc'
filt_width = 12.

data = xr.open_dataset(file_in,chunks={'time_counter':1})

In [ ]:
# Testing high-pass filter stuff:
file_in_day = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'
file_in_mon = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_mon.nc'
file_out = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_mon_hp.nc'
filt_width = 12.

ds = ptools.calc_zhp_std_variables(file_in_day,file_in_mon,file_out,filt_width)